In [30]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [31]:
import pandas as pd 
import numpy as np

from math import sqrt
from numpy import concatenate
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot as plt

from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

In [32]:
days_to_predict = int(input('How many days from today do you want to predict? '))
print(days_to_predict)

How many days from today do you want to predict? 3
3


In [33]:
import datetime
today =  datetime.date.today()
year_today = today.year
month_today = today.month
day_today = today.day

future = datetime.date.today() + datetime.timedelta(days=days_to_predict)
year_future = future.year
month_future = future.month
day_future = future.day

future_df = (pd.DataFrame(columns=['NULL'],
                  index=pd.date_range(today, future,
                                      freq='30T'))
       .between_time('09:30','15:30')
       .index.strftime('%Y-%m-%d %H:%M:%S')
       .tolist()
)

future_df = pd.DataFrame(future_df, columns=['created_at'])
future_df['created_at'] = future_df["created_at"].apply(pd.to_datetime)
future_df = future_df[future_df['created_at'].dt.weekday < 5]
future_df = future_df.set_index('created_at')
future_df['Close'] = np.nan
future_df['Comp'] = np.nan
future_df

,Close,Comp
created_at,,
2021-03-24 09:30:00,NaN,NaN
2021-03-24 10:00:00,NaN,NaN
2021-03-24 10:30:00,NaN,NaN
2021-03-24 11:00:00,NaN,NaN
2021-03-24 11:30:00,NaN,NaN
2021-03-24 12:00:00,NaN,NaN
2021-03-24 12:30:00,NaN,NaN
2021-03-24 13:00:00,NaN,NaN
2021-03-24 13:30:00,NaN,NaN


In [34]:
predict_rows = len(future_df)
print(predict_rows)

39


In [35]:
final = pd.read_csv('/content/drive/MyDrive/FA PROJECT 2021/LSTM/SIGL/final.csv') 
final = final.set_index('Datetime')
final.head()

,Close,Comp
Datetime,,
2021-01-21 09:30:00,6.86,0.0000
2021-01-21 13:30:00,6.28,0.1480
2021-01-21 14:30:00,6.41,-0.1480
2021-01-21 15:00:00,6.51,0.3612
2021-01-22 12:00:00,6.23,-0.1174


In [36]:
#Scaling for dataframes with 2 columns
# Feature Scaling for fast training of neural networks
from sklearn.preprocessing import StandardScaler, MinMaxScaler
 
sc = MinMaxScaler()
 
DataScaler = sc.fit(final)
X = DataScaler.transform(final)

In [37]:
#Scaling for dataframes with 1 column
# Feature Scaling for fast training of neural networks
from sklearn.preprocessing import StandardScaler, MinMaxScaler
 
df_price = final.drop(columns='Comp')

sc_y = MinMaxScaler()
 
DataScaler_y = sc_y.fit(df_price)
Y = DataScaler_y.transform(df_price)

In [38]:
# split into samples
X_samples = list()
y_samples = list()
 
NumerOfRows = len(X)
TimeSteps= 26  # Price Prediction for n days is based on last how many rows of data, 26 = 2 days
 
# Iterate thru the values to create combinations
for i in range(TimeSteps , NumerOfRows , 1):
    x_sample = X[i-TimeSteps:i]
    y_sample = X[i]
    X_samples.append(x_sample)
    y_samples.append(y_sample)

In [39]:
X_data=np.array(X_samples)
X_data.shape

(65, 26, 2)

In [40]:
################################################
# Reshape the Input as a 3D (number of samples, Time Steps, Features)
X_data=np.array(X_samples)
X_data=X_data.reshape(X_data.shape[0],X_data.shape[1], -1)
print('\n#### Input Data shape ####')
print(X_data.shape)
 
# We do not reshape y as a 3D data  as it is supposed to be a single column only
y_data=np.array(y_samples)
y_data=y_data.reshape(y_data.shape[0], -1)
print('\n#### Output Data shape ####')
print(y_data.shape)


#### Input Data shape ####
(65, 26, 2)

#### Output Data shape ####
(65, 2)


In [41]:
# Choosing the number of testing data records
TestingRecords= round(0.2 * y_data.shape[0]) #take 20% of the data for testing

# Splitting the data into train and test
#https://datascience.stackexchange.com/questions/30762/how-to-predict-the-future-values-of-time-horizon-with-keras
X_train=X_data[:-TestingRecords+predict_rows]
X_test=X_data[-TestingRecords:]
y_train=y_data[:-TestingRecords+predict_rows]
y_test=y_data[-TestingRecords:]
 
############################################
 
# Printing the shape of training and testing
print('\n#### Training Data shape ####')
print(X_train.shape)
print(y_train.shape)
print('\n#### Testing Data shape ####')
print(X_test.shape)
print(y_test.shape)


#### Training Data shape ####
(26, 26, 2)
(26, 2)

#### Testing Data shape ####
(13, 26, 2)
(13, 2)


In [42]:
# Defining Input shapes for LSTM
TimeSteps=X_train.shape[1]
TotalFeatures=X_train.shape[2]
print("Number of TimeSteps:", TimeSteps)
print("Number of Features:", TotalFeatures)

Number of TimeSteps: 26
Number of Features: 2


In [43]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
 
# Initialising the RNN
regressor = Sequential()
 
# Adding the First input hidden layer and the LSTM layer
# return_sequences = True, means the output of every time step to be shared with hidden next layer
regressor.add(LSTM(units = 10, activation = 'tanh', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))
 
# Adding the Second Second hidden layer and the LSTM layer
regressor.add(LSTM(units = 5, activation = 'tanh', input_shape = (TimeSteps, TotalFeatures), return_sequences=True))
 
# Adding the Second Third hidden layer and the LSTM layer
regressor.add(LSTM(units = 5, activation = 'tanh', return_sequences=False ))
 
# Adding the output layer
regressor.add(Dense(units = 1))
 
# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
 
##################################################
 
import time
# Measuring the time taken by the model to train
StartTime=time.time()
 
# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 5, epochs = 50)
 
EndTime=time.time()
print("## Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ##')

Epoch 1/50
6/6 [==============================] - 5s 28ms/step - loss: 0.3587
Epoch 2/50
6/6 [==============================] - 0s 29ms/step - loss: 0.2541
Epoch 3/50
6/6 [==============================] - 0s 32ms/step - loss: 0.1588
Epoch 4/50
6/6 [==============================] - 0s 30ms/step - loss: 0.1289
Epoch 5/50
6/6 [==============================] - 0s 29ms/step - loss: 0.1064
Epoch 6/50
6/6 [==============================] - 0s 30ms/step - loss: 0.0618
Epoch 7/50
6/6 [==============================] - 0s 29ms/step - loss: 0.0703
Epoch 8/50
6/6 [==============================] - 0s 32ms/step - loss: 0.0572
Epoch 9/50
6/6 [==============================] - 0s 28ms/step - loss: 0.0794
Epoch 10/50
6/6 [==============================] - 0s 29ms/step - loss: 0.0700
Epoch 11/50
6/6 [==============================] - 0s 32ms/step - loss: 0.0703
Epoch 12/50
6/6 [==============================] - 0s 31ms/step - loss: 0.0632
Epoch 13/50
6/6 [==============================] - 0s 33ms/st

In [44]:
# Making predictions on test data
predicted_Price = regressor.predict(X_test)
predicted_Price = DataScaler_y.inverse_transform(predicted_Price)

# Getting the original price values for testing data
orig=y_test
orig=DataScaler_y.inverse_transform(y_test)

# Accuracy of the predictions
print('Accuracy:', 100 - (100*(abs(orig-predicted_Price)/orig)).mean())

Accuracy: 74.2295693818238


In [45]:
# Generating predictions on full data
TrainPredictions=DataScaler_y.inverse_transform(regressor.predict(X_train))
TestPredictions=DataScaler_y.inverse_transform(regressor.predict(X_test))

FullDataPredictions=np.append(TrainPredictions, TestPredictions)
FullDataOrig=final[TimeSteps:]

FullDataOrig = FullDataOrig.append(future_df)
FullDataOrig['Predicted_Close'] = FullDataPredictions
FullDataOrig

ValueError: ignored

In [ ]:
FullDataOrig.index=FullDataOrig.index.astype(str)
FullDataOrig

In [ ]:
# plotting the full data
plt.plot(FullDataOrig['Close'] , color = 'lightblue', label = 'Original Price')
plt.plot(FullDataOrig['Predicted_Close'] , color = 'blue', label = 'Predicted Price')


plt.title('Stock Price Predictions')
plt.xlabel('Trading Datetime')
plt.ylabel('Stock Price')
plt.legend()
fig=plt.gcf()
fig.set_figwidth(20)
fig.set_figheight(8)
plt.show()